In [16]:
import os, glob
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydicom import dcmread
from PIL import Image

In [17]:
# ─── POINT EVERYTHING AT Extra Storage ───────────────────────────────────────
DRIVE_ROOT    = "/Volumes/Extra Storage"
data_dir      = os.path.join(DRIVE_ROOT, "test_data", "test_data")         # original DICOMs / images
results_dir1  = os.path.join(DRIVE_ROOT, "local_ADS_data", "necessary_sufficient")    # *_final_mask.pt & *_loss.png
results_dir2  = os.path.join(DRIVE_ROOT, "local_ADS_data", "gradcam_results")  # grad-cam PNGs/JPGs named <prefix>.*
output_dir    = os.path.join(DRIVE_ROOT, "local_ADS_data", "combined_results")
input_csv     = os.path.join(DRIVE_ROOT, "local_ADS_data", "best_model_pred_final.csv")        # CSV with labels
csv_search_col = "dicom_path"                 # column that contains the prefix in its string
csv_label_col  = "true_label"            # column holding your ground-truth label
# ─────────────────────────────────────────────────────────────────────────────

In [18]:
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(input_csv)

In [19]:

def load_gradcam(path):
    return np.array(Image.open(path))

def load_mask(path):
    data = torch.load(path, map_location="cpu")
    m = data.get("final_mask", data) if isinstance(data, dict) else data
    return m.detach().cpu().numpy().squeeze()

def load_original(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".dcm":
        ds  = dcmread(path, force=True)
        arr = ds.pixel_array.astype(np.float32)
        return (arr - arr.min()) / (np.ptp(arr) + 1e-6)
    else:
        return np.array(Image.open(path).convert("L"), dtype=np.float32) / 255.0

def load_metrics(path):
    return np.array(Image.open(path).convert("RGB"))




In [ ]:
# collect all prefixes from data_dir
prefixes = [os.path.splitext(os.path.basename(f))[0]
            for f in glob.glob(os.path.join(data_dir, "*"))]

for prefix in prefixes:
    # 1) lookup ground-truth label
    row   = df[df[csv_search_col].str.contains(prefix, na=False)]
    label = row.iloc[0][csv_label_col] if not row.empty else "UNKNOWN"

    # 2) define paths
    gradcam_glob = glob.glob(os.path.join(results_dir2, f"{prefix}.*"))
    mask_path    = os.path.join(results_dir1, f"{prefix}_final_mask.pt")
    metrics_path = os.path.join(results_dir1, f"{prefix}_metrics.png")
    orig_glob    = glob.glob(os.path.join(data_dir,       f"{prefix}.*"))

    if not (gradcam_glob and orig_glob and os.path.isfile(mask_path)):
        print(f"[SKIP] missing data for '{prefix}'")
        continue

    # 3) load each element
    gradcam = load_gradcam(gradcam_glob[0])
    orig    = load_original(orig_glob[0])
    mask    = load_mask(mask_path)
    metrics = load_metrics(metrics_path) if os.path.isfile(metrics_path) else None

    # 4) resize mask to match orig if necessary
    if mask.shape != orig.shape:
        m = Image.fromarray((mask*255).astype(np.uint8))
        m = m.resize((orig.shape[1], orig.shape[0]), Image.NEAREST)
        mask = np.array(m, dtype=np.float32) / 255.0

    # 5) plot 1×4 with updated figure title
    fig, axs = plt.subplots(1, 4, figsize=(18,4))
    fig.suptitle(f"DICOM ID: {prefix} | True Label: {label}", fontsize=18, y=1.02)

    axs[0].imshow(gradcam)
    axs[0].set_title("Grad-CAM")
    axs[1].imshow(mask, cmap="viridis", vmin=0, vmax=1)
    axs[1].set_title("Final Mask")
    axs[2].imshow(orig, cmap="gray", vmin=0, vmax=1)
    axs[2].imshow(mask, cmap="jet", alpha=0.3)
    axs[2].set_title("Overlay")
    if metrics is not None:
        axs[3].imshow(metrics)
        axs[3].set_title("Metrics")
    else:
        axs[3].axis("off")
        axs[3].set_title("No Metrics")

    for ax in axs:
        ax.axis("off")

    out_path = os.path.join(output_dir, f"{prefix}_summary.png")
    plt.savefig(out_path, bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    print(f"→ saved {out_path}")


→ saved /Volumes/Extra Storage/local_ADS_data/combined_results/b9a08a39-c53ad784-99673387-d9140a2f-cbc1dbde_summary.png
→ saved /Volumes/Extra Storage/local_ADS_data/combined_results/cf8e40e4-13142780-e760eaf4-54a31990-8b748ee4_summary.png
→ saved /Volumes/Extra Storage/local_ADS_data/combined_results/5d93739f-badcdac7-b7414fdb-8db69418-9838f396_summary.png
→ saved /Volumes/Extra Storage/local_ADS_data/combined_results/b7fc3517-dd72ab6b-fe8826e7-93b3186a-cbc6f8ec_summary.png
